<a href="https://colab.research.google.com/github/hojuna/Black-box_Optimization_dacon/blob/main/blanding_Quantile_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Quantile_regression을 통해서 0.95, 0.9를 기준으로 학습하는데

0.95_model=0.8
0.9_model=0.2
정도로 블랜딩해서 하나의 모델로 만든다..

여기서 중요한것은 하이퍼파라미터 튜닝을 어떻게 할것인가?, 또한 블랜딩 비율에 대한 고민도 필요하다.

최종본 제출안함.. 해봤자 0.72예상

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Reduce Memory Usage
# reference : https://www.kaggle.com/code/arjanso/reducing-dataframe-memory-size-by-65 @ARJANGROEN

def reduce_memory_usage(df):

    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype.name
        if ((col_type != 'datetime64[ns]') & (col_type != 'category')):
            if (col_type != 'object'):
                c_min = df[col].min()
                c_max = df[col].max()

                if str(col_type)[:3] == 'int':
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)

                else:
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float16)
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    else:
                        pass
            else:
                df[col] = df[col].astype('category')
    mem_usg = df.memory_usage().sum() / 1024**2
    print("Memory usage became: ",mem_usg," MB")

    return df

In [3]:
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV,cross_val_score
from sklearn.model_selection import RandomizedSearchCV


# Load the data
train_df_org = pd.read_csv('/content/drive/MyDrive/dacon/2024_8_data/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/dacon/2024_8_data/test.csv')

In [4]:
train_df_org_reduced = reduce_memory_usage(train_df_org)
test_df_reduced = reduce_memory_usage(test_df)

Memory usage of dataframe is 3.98 MB
Memory usage became:  2.38531494140625  MB
Memory usage of dataframe is 0.46 MB
Memory usage became:  0.27829742431640625  MB


In [5]:
# df_shuffled= train_df_org
df_shuffled= train_df_org_reduced

# 균등하게 나누기 위해 임의로 데이터를 8:2로 분할
split_index = int(len(df_shuffled) * 0.8)
train_df, valiation_df = df_shuffled[:split_index], df_shuffled[split_index:]


In [6]:

def val_score(model, valiation_df):
  id_y=pd.DataFrame()

  top_5_percent_valiation_df = valiation_df.sort_values(by='y').reset_index(drop=True)[-int(0.05 * len(valiation_df)):]
  #id만 분리 valiation_df, y값 제거
  X = valiation_df.iloc[:, 1:-1]
  id_y['ID']=valiation_df.iloc[:,0]

  #valiation_df y값 예측
  y_pred=model.predict(X)
  id_y['y_pred']=y_pred

  # print(id_x.head)
  # #id 복원

  #상위 10% valiation_df 구하기
  top_10_percent_pred_valiation=id_y.sort_values(by='y_pred').reset_index(drop=True)[-int(0.10 * len(id_y)):]

  # ID 값에서 공백 제거
  top_5_percent_valiation_df['ID'] = top_5_percent_valiation_df['ID'].str.strip()
  top_10_percent_pred_valiation['ID'] = top_10_percent_pred_valiation['ID'].str.strip()

  #top_5_percent_valiation_df 중  상위 10%에 포함되는 비율을 구해서 반환
  return top_5_percent_valiation_df['ID'].isin(top_10_percent_pred_valiation['ID']).sum()/len(top_5_percent_valiation_df)

In [7]:
from sklearn.preprocessing import StandardScaler

X = train_df.iloc[:, 1:-1]  # Features
y = train_df['y']  # Target

# Standardize the features
scaler_x = StandardScaler()
scaler_y = StandardScaler()

X_scaled = scaler_x.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))



In [8]:

from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import QuantileRegressor
import joblib

# 하이퍼파라미터 범위를 설정합니다.
param_distributions = {
    'quantile': [0.9],
    'alpha':  [0, 0.1, 0.4, 0.7, 0.9],
    'fit_intercept': [True, False],
    'solver': ['highs', 'interior-point', 'soplex'],

}

# QuantileRegressor 모델 생성
qr_model = QuantileRegressor()

# RandomizedSearchCV 생성
random_search_90 = RandomizedSearchCV(estimator=qr_model, param_distributions=param_distributions, n_iter=30, cv=3)

# 학습 데이터에 대해 최적의 하이퍼파라미터를 찾습니다.
random_search_90.fit(X_scaled, y_scaled)

# 최적의 하이퍼파라미터 조합과 성능을 출력합니다.
print("Best Parameters:", random_search_90.best_params_)
print("Best Score:", random_search_90.best_score_)

val_score(random_search_90.best_estimator_,valiation_df)

joblib.dump(random_search_90.best_estimator_, '/content/drive/MyDrive/dacon/model/Quantile/qr_model_90.pkl')

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exam

Best Parameters: {'solver': 'highs', 'quantile': 0.9, 'fit_intercept': False, 'alpha': 0}
Best Score: 0.6011130578601361


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:458: UserWarning: X has feature names, but QuantileRegressor was fitted without feature names
  warnings.warn(


['/content/drive/MyDrive/dacon/model/Quantile/qr_model_90.pkl']

In [15]:

val_score(random_search_90.best_estimator_,valiation_df)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:458: UserWarning: X has feature names, but QuantileRegressor was fitted without feature names
  warnings.warn(


0.0

In [10]:
# QuantileRegressor 모델 생성
qr_model = QuantileRegressor()

param_distributions['quantile']=[0.95]

# RandomizedSearchCV 생성
random_search_95 = RandomizedSearchCV(estimator=qr_model, param_distributions=param_distributions, n_iter=30, cv=3)

# 학습 데이터에 대해 최적의 하이퍼파라미터를 찾습니다.
random_search_95.fit(X_scaled, y_scaled)

# 최적의 하이퍼파라미터 조합과 성능을 출력합니다.
print("Best Parameters:", random_search_95.best_params_)
print("Best Score:", random_search_95.best_score_)

val_score(random_search_95.best_estimator_,valiation_df)

joblib.dump(random_search_95.best_estimator_, '/content/drive/MyDrive/dacon/model/Quantile/qr_model_95.pkl')

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exam

Best Parameters: {'solver': 'highs', 'quantile': 0.95, 'fit_intercept': False, 'alpha': 0}
Best Score: 0.6011103057740281


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:458: UserWarning: X has feature names, but QuantileRegressor was fitted without feature names
  warnings.warn(


['/content/drive/MyDrive/dacon/model/Quantile/qr_model_95.pkl']

In [43]:
from sklearn.preprocessing import MinMaxScaler
def blanding_model_predict(model_1, model_2,test_df):


  # X_test = test_df.iloc[:, 1:]  # Features
  model_1_pred=model_1.predict(test_df)
  model_2_pred=model_2.predict(test_df)

      # 정규화
  # scaler = MinMaxScaler(feature_range=(0, 100))
  # model_1_pred = scaler.fit_transform(model_1_pred.reshape(-1, 1)).flatten()
  # model_2_pred = scaler.fit_transform(model_2_pred.reshape(-1, 1)).flatten()
      # 블렌딩 가중치 적용
  blended_pred = model_1_pred * 0.7 + model_2_pred * 0.3

  # 결과 클리핑
  # blended_pred = np.clip(blended_pred, 0, 100)

  return blended_pred

In [12]:
valiation_df.head()

,ID,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,x_10,y
32094,TRAIN_32094,1.006836,-2.013672,1.228516,0.905273,-0.259766,-1.786133,0.468506,-0.145874,0.625000,0.313965,0.077881,82.6875
32095,TRAIN_32095,1.111328,-2.494141,1.207031,0.872559,-0.250244,-1.811523,0.493896,-0.180298,0.676270,0.264648,0.095581,85.1875
32096,TRAIN_32096,1.043945,-2.503906,1.198242,0.790527,-0.264404,-1.791016,0.495117,-0.190674,0.688965,0.254639,0.093628,82.4375
32097,TRAIN_32097,1.073242,-2.484375,1.201172,0.844238,-0.264404,-1.799805,0.500000,-0.192505,0.684082,0.257812,0.095032,84.0000
32098,TRAIN_32098,1.083984,-2.439453,1.209961,0.902344,-0.250977,-1.816406,0.481689,-0.175537,0.666992,0.280762,0.095764,84.7500


In [45]:

  id_y=pd.DataFrame()

  top_5_percent_valiation_df = valiation_df.sort_values(by='y').reset_index(drop=True)[-int(0.05 * len(valiation_df)):]
  #id만 분리 valiation_df, y값 제거

  X = valiation_df.iloc[:, 1:-1]
  X_scaled = scaler_x.transform(X)
  id_y['ID']=valiation_df.iloc[:,0]

  #valiation_df y값 예측
  y_pred=blanding_model_predict(random_search_95.best_estimator_,random_search_90.best_estimator_,X_scaled)
  id_y['y_pred']=scaler_y.inverse_transform(y_pred.reshape(-1, 1)).flatten()

  # print(id_x.head)
  # #id 복원

  #상위 10% valiation_df 구하기
  top_10_percent_pred_valiation=id_y.sort_values(by='y_pred').reset_index(drop=True)[-int(0.10 * len(id_y)):]

  # ID 값에서 공백 제거
  top_5_percent_valiation_df['ID'] = top_5_percent_valiation_df['ID'].str.strip()
  top_10_percent_pred_valiation['ID'] = top_10_percent_pred_valiation['ID'].str.strip()

  #top_5_percent_valiation_df 중  상위 10%에 포함되는 비율을 구해서 반환
  print(top_5_percent_valiation_df['ID'].isin(top_10_percent_pred_valiation['ID']).sum()/len(top_5_percent_valiation_df))

0.8778054862842892


In [44]:
# Predict on test data
X_test = test_df_reduced.iloc[:, 1:]
X_test_scaled = scaler_x.transform(X_test)

y_pred_scaled = blanding_model_predict(random_search_95.best_estimator_,random_search_90.best_estimator_,X_test_scaled)

# y_pred = random_search_95.best_estimator_.predict(X_test)
# y_pred = random_search_90.best_estimator_.predict(X_test)

y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()

# Identify top 5% of predicted values
threshold = np.percentile(y_pred, 95)
top_5_percent_mask = y_pred >= threshold

# Create submission file
submission_df = pd.read_csv("/content/drive/MyDrive/dacon/2024_8_data/submission/sample_submission.csv")
submission_df['y'] = y_pred
submission_df.to_csv("/content/drive/MyDrive/dacon/2024_8_data/submission/blanding_QR_submission.csv", index=False)

print(f"Top 5% threshold: {threshold:.4f}")
print(f"Number of samples in top 5%: {sum(top_5_percent_mask)}")

Top 5% threshold: 92.8866
Number of samples in top 5%: 250
